In [1]:
import glob, os
print("Mounted pcosgen_out paths:", glob.glob("/kaggle/input/*/pcosgen_out"))
print("All inputs under /kaggle/input:", os.listdir("/kaggle/input"))


Mounted pcosgen_out paths: ['/kaggle/input/pcos-pic/pcosgen_out']
All inputs under /kaggle/input: ['pcos-pic', 'pcos-picture']


In [2]:
# === Cell A: Environment locks & imports ===
!pip -q uninstall -y albumentations albucore
!pip -q install --no-deps albucore==0.0.20 albumentations==1.4.16 timm==0.9.16

import os, glob, json, math, warnings, time, hashlib
from pathlib import Path

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import (
    roc_auc_score, f1_score, accuracy_score, confusion_matrix,
    roc_curve, auc, precision_recall_curve, ConfusionMatrixDisplay
)

import timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.6/214.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.3 MB/s eta 0:00:00:00:01


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
# === Cell B: Config, copy artifacts, rebuild df ===
class CFG:
    BASE = "/kaggle/input/pcos-picture"  # dataset with images & labels
    OUT_DIR = "/kaggle/working/pcosgen_out"
    img_size = 448
    batch_size = 24
    num_workers = 4
    backbone = "convnext_tiny.fb_in22k"

# --- copy previous outputs (pcosgen_out) from your prior Version input ---
cand = glob.glob("/kaggle/input/*/pcosgen_out")
assert len(cand) > 0, "Could not find previous pcosgen_out under /kaggle/input/*/pcosgen_out. Add your prior version as an Input."
SRC_OUT = cand[0]
!rm -rf /kaggle/working/pcosgen_out
!cp -r "$SRC_OUT" /kaggle/working/pcosgen_out
print("Restored artifacts from:", SRC_OUT)

# --- rebuild df from pcos-picture ---
def load_labels_from_table(path):
    ext = os.path.splitext(path)[1].lower()
    df_lab = pd.read_excel(path, sheet_name=0) if ext in [".xlsx", ".xls"] else pd.read_csv(path)
    df_lab.columns = [str(c).strip().lower() for c in df_lab.columns]
    img_col = next((c for c in ["imagepath","image","filename","file","img","name"] if c in df_lab.columns), None)
    lab_col = next((c for c in ["healthy","label","class","target","y"] if c in df_lab.columns), None)
    assert img_col and lab_col, "Image or Label column not found."
    name = (df_lab[img_col].astype(str).str.strip().apply(os.path.basename).str.lower())
    lab = (df_lab[lab_col].astype(str).str.strip().str.lower()
           .map({"1":1,"0":0,"healthy":1,"unhealthy":0,"h":1,"u":0}).astype(int))
    table = pd.DataFrame({"image": name, "label": lab}).dropna().drop_duplicates("image", keep="last")
    return dict(zip(table["image"], table["label"]))

def list_images(dir_path):
    exts = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")
    return sorted([str(p) for p in Path(dir_path).glob("*") if str(p).lower().endswith(exts)])

train_path = Path(CFG.BASE) / "PCOSGen-train (1)" / "PCOSGen-train"
TRAIN_DIR = str(train_path / "images")
LABELS_FILE = str(train_path / "class_label.xlsx")

train_files = list_images(TRAIN_DIR)
bn_to_path = {os.path.basename(p).lower(): p for p in train_files}
label_map = load_labels_from_table(LABELS_FILE)
df = pd.DataFrame([(bn_to_path[bn], lab) for bn, lab in label_map.items() if bn in bn_to_path],
                  columns=["path","y"]).reset_index(drop=True)
print("Train images in df:", len(df))


Restored artifacts from: /kaggle/input/pcos-pic/pcosgen_out
Train images in df: 3200


In [4]:
# === Cell C: Transforms, dataset, model helpers ===
IMAGENET_MEAN, IMAGENET_STD = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)

valid_tfms = A.Compose([
    A.LongestMaxSize(max_size=CFG.img_size),
    A.PadIfNeeded(CFG.img_size, CFG.img_size, border_mode=cv2.BORDER_REFLECT),
    A.CLAHE(p=1.0),
    A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
    ToTensorV2(),
])

class USDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = cv2.imread(row.path, cv2.IMREAD_UNCHANGED)
        if img is None:
            img = np.zeros((CFG.img_size, CFG.img_size), dtype=np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) if img.ndim == 2 else cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x = self.transform(image=img)["image"]
        y = int(row.y)
        return x, torch.tensor([y], dtype=torch.float32)

def compute_metrics(y_true, y_prob, thr=0.5):
    y_true = np.asarray(y_true).astype(int)
    y_pred = (y_prob >= thr).astype(int)
    auc_v = roc_auc_score(y_true, y_prob) if len(np.unique(y_true))>1 else 0.5
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    spec = tn / (tn+fp+1e-9)
    rec  = tp / (tp+fn+1e-9)
    return dict(AUC=auc_v, F1=f1, ACC=acc, SPEC_unhealthy=spec, RECALL_healthy=rec)

def best_threshold(y_true, y_prob, metric="J", n=1001):
    thrs = np.linspace(0.0, 1.0, n)
    best_t, best_v = 0.5, -1.0
    for t in thrs:
        m = compute_metrics(y_true, y_prob, thr=t)
        v = m["RECALL_healthy"] + m["SPEC_unhealthy"] - 1.0  # Youden's J
        if v > best_v:
            best_v, best_t = v, t
    return float(best_t), float(best_v)

def find_threshold_for_spec(y_true, y_prob, target_spec=0.80):
    for t in np.linspace(0, 1, 1001):
        if compute_metrics(y_true, y_prob, thr=t)['SPEC_unhealthy'] >= target_spec:
            return float(t)
    return 0.5

class PCOSNet(nn.Module):
    def __init__(self, backbone=CFG.backbone, pretrained=False):
        super().__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained, num_classes=1, in_chans=3, drop_rate=0.2)
    def forward(self, x): return self.backbone(x).squeeze(1)

def load_ckpt_into_base(ckpt, backbone, device):
    model = PCOSNet(backbone, pretrained=False).to(device)
    sd = ckpt["state_dict"]
    if any(k.startswith("module.") for k in sd.keys()):
        sd = {k.replace("module.", "", 1): v for k, v in sd.items()}
    model.load_state_dict(sd, strict=True)
    model.eval()
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

@torch.no_grad()
def tta_logits(model, xb, do_transpose=True):
    outs = []
    outs.append(model(xb))
    outs.append(model(torch.flip(xb, dims=[-1])))
    outs.append(model(torch.flip(xb, dims=[-2])))
    outs.append(model(torch.flip(torch.flip(xb, [-1]), [-2])))
    if do_transpose:
        xbt = xb.transpose(-1, -2).contiguous()
        outs.append(model(xbt))
        outs.append(model(torch.flip(xbt, dims=[-1])))
        outs.append(model(torch.flip(xbt, dims=[-2])))
        outs.append(model(torch.flip(torch.flip(xbt, [-1]), [-2])))
    return sum(outs) / len(outs)


In [5]:
# === Cell R1: restore thresholds from previous run ===
with open(f"{CFG.OUT_DIR}/thresholds.json") as f:
    THR = json.load(f)

global_t_temp = float(THR["global_t_temp"])
thr_j  = float(THR["thr_j"])
thr_s80= float(THR["thr_s80"])
thr_85 = float(THR["thr_85"])
print("Restored thresholds:", {k: THR[k] for k in ["global_t_temp","thr_j","thr_s80","thr_85"]})


Restored thresholds: {'global_t_temp': 0.581762619911359, 'thr_j': 0.463, 'thr_s80': 0.483, 'thr_85': 0.46346965432167053}


In [7]:
# === Cell 17R-FAST: Accurate latency (model-only + optional preprocessing) ===
import os, glob, time, json, math, numpy as np
import torch, torch.nn as nn

# ---- safe fallbacks ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
OUT = getattr(CFG, "OUT_DIR", "/kaggle/working/pcosgen_out")
IMG = getattr(CFG, "img_size", 448)
USE_TTA = False   # fair vs tabular; set True if you also want TTA timing
BATCHES = [1, 8, 32]  # report online latency (1) and throughput (8, 32)
WARMUP = 20
ITERS  = 100      # increase to 200+ for even tighter error bars

# ---- helpers ----
def _forward(model, xb, use_tta=False):
    if use_tta and "tta_logits" in globals():
        return tta_logits(model, xb)
    return model(xb)

@torch.no_grad()
def time_one_setting(model, bs, iters=ITERS, warmup=WARMUP, use_tta=False):
    model.eval()
    x = torch.randn(bs, 3, IMG, IMG, device=device)

    use_amp = (device.type == "cuda")
    # warmup
    for _ in range(warmup):
        with torch.cuda.amp.autocast(enabled=use_amp):
            _ = _forward(model, x, use_tta)
        if device.type == "cuda":
            torch.cuda.synchronize()

    # timing (CUDA events for precision)
    if device.type == "cuda":
        starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
        times = []
        for _ in range(iters):
            starter.record()
            with torch.cuda.amp.autocast(enabled=use_amp):
                _ = _forward(model, x, use_tta)
            ender.record()
            torch.cuda.synchronize()
            times.append(starter.elapsed_time(ender) / 1000.0)  # seconds
    else:
        times = []
        for _ in range(iters):
            t0 = time.perf_counter()
            _ = _forward(model, x, use_tta)
            t1 = time.perf_counter()
            times.append(t1 - t0)

    times = np.array(times, dtype=float)
    per_img_ms = times.mean() * 1000.0 / bs
    result = {
        "batch_size": int(bs),
        "mean_s": float(times.mean()),
        "median_s": float(np.median(times)),
        "p90_s": float(np.percentile(times, 90)),
        "p99_s": float(np.percentile(times, 99)),
        "per_image_ms": float(per_img_ms),
        "throughput_img_per_s": float(bs / times.mean())
    }
    return result

# ---- load best checkpoint by stored AUC ----
cands = sorted(glob.glob(f"{OUT}/model_fold*.pt"))
assert len(cands) > 0, "No checkpoints found in OUT_DIR."
best_auc, best_p = -1.0, None
for p in cands:
    meta = torch.load(p, map_location="cpu", weights_only=False)
    a = float(meta.get("auc", -1.0))
    if a > best_auc: best_auc, best_p = a, p

ckpt = torch.load(best_p, map_location=device, weights_only=False)
model = load_ckpt_into_base(ckpt, ckpt["backbone"], device)
# Optional: use both GPUs for max throughput
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
print(f"[latency] Using {os.path.basename(best_p)} (AUC meta={best_auc:.4f}) on {device}")

# ---- run timings (no-TTA and optionally TTA) ----
summary = {"device": str(device), "img_size": int(IMG), "tta": bool(USE_TTA), "results": []}
for bs in BATCHES:
    res_no = time_one_setting(model, bs, use_tta=False)
    entry = {"mode": f"no_TTA_bs{bs}", **res_no}
    summary["results"].append(entry)
    print(entry)

if USE_TTA:
    for bs in BATCHES:
        res_tta = time_one_setting(model, bs, use_tta=True)
        entry = {"mode": f"TTA_bs{bs}", **res_tta}
        summary["results"].append(entry)
        print(entry)

# ---- optional: preprocessing (Albumentations valid_tfms) timing on a small sample ----
try:
    import cv2, random
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    assert "valid_tfms" in globals() and "df" in globals() and len(df) > 0
    n = min(200, len(df))  # small, just to estimate CPU preprocessing cost
    paths = df.sample(n=n, random_state=123).path.tolist()
    t0 = time.perf_counter()
    for p in paths:
        img = cv2.imread(p, cv2.IMREAD_UNCHANGED)
        if img is None: continue
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) if img.ndim == 2 else cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        _ = valid_tfms(image=img)["image"]  # tensor (unused)
    t1 = time.perf_counter()
    prep_ms_per_img = (t1 - t0) * 1000.0 / max(1, n)
    summary["preprocessing_ms_per_image_CPU"] = float(prep_ms_per_img)
    print({"preprocessing_ms_per_image_CPU": prep_ms_per_img})
except Exception as e:
    print(f"[prep] skipped preprocessing timing: {e}")

# ---- save
os.makedirs(OUT, exist_ok=True)
with open(f"{OUT}/latency_report_fast.json", "w") as f:
    json.dump(summary, f, indent=2)
print(f"[latency] Saved -> {OUT}/latency_report_fast.json")


[latency] Using model_fold1.pt (AUC meta=0.8574) on cuda


/tmp/ipykernel_36/3227563125.py:28: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):
/tmp/ipykernel_36/3227563125.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


{'mode': 'no_TTA_bs1', 'batch_size': 1, 'mean_s': 0.03196150320053101, 'median_s': 0.028668736457824705, 'p90_s': 0.030194438362121583, 'p99_s': 0.0344940617370621, 'per_image_ms': 31.96150320053101, 'throughput_img_per_s': 31.28763981236608}
{'mode': 'no_TTA_bs8', 'batch_size': 8, 'mean_s': 0.06587895389556886, 'median_s': 0.06557700729370117, 'p90_s': 0.06621220245361328, 'p99_s': 0.07705201400756839, 'per_image_ms': 8.234869236946107, 'throughput_img_per_s': 121.4348365743873}
{'mode': 'no_TTA_bs32', 'batch_size': 32, 'mean_s': 0.1911335923767089, 'median_s': 0.1894207992553711, 'p90_s': 0.19182490692138673, 'p99_s': 0.1954764141845714, 'per_image_ms': 5.972924761772154, 'throughput_img_per_s': 167.42216583744514}
{'preprocessing_ms_per_image_CPU': 7.469219194999823}
[latency] Saved -> /kaggle/working/pcosgen_out/latency_report_fast.json


In [10]:
import json, pprint, os
p = "/kaggle/working/pcosgen_out/latency_report_fast.json"
with open(p) as f: rpt = json.load(f)

# pull the rows you need
r1  = [r for r in rpt["results"] if r["mode"]=="no_TTA_bs1"][0]
r8  = [r for r in rpt["results"] if r["mode"]=="no_TTA_bs8"][0]
r32 = [r for r in rpt["results"] if r["mode"]=="no_TTA_bs32"][0]
prep = rpt.get("preprocessing_ms_per_image_CPU", None)

print(f"ms/img (no-TTA, bs=1): {r1['per_image_ms']:.2f} ms")
print(f"imgs/s (no-TTA, bs=8): {r8['throughput_img_per_s']:.2f} img/s")
print(f"imgs/s (no-TTA, bs=32): {r32['throughput_img_per_s']:.2f} img/s")
if prep is not None:
    print(f"End-to-end (bs=1, approx): {r1['per_image_ms']+prep:.2f} ms/img")


ms/img (no-TTA, bs=1): 31.96 ms
imgs/s (no-TTA, bs=8): 121.43 img/s
imgs/s (no-TTA, bs=32): 167.42 img/s
End-to-end (bs=1, approx): 39.43 ms/img


In [8]:
# === Cell 19R2: Build HTML appendix from saved files ===
import base64
from datetime import datetime

def _img_data_uri(p):
    try:
        with open(p, 'rb') as f: return 'data:image/png;base64,' + base64.b64encode(f.read()).decode('ascii')
    except Exception: return None

def build_html_appendix_fileonly(out_html=f"{CFG.OUT_DIR}/appendix.html"):
    def try_json(p):
        try:
            with open(p) as f: return json.load(f)
        except Exception: return {}

    thr = try_json(f"{CFG.OUT_DIR}/thresholds.json")
    rep = try_json(f"{CFG.OUT_DIR}/latency_report.json")
    metrics_ci = try_json(f"{CFG.OUT_DIR}/oof_metrics_ci.json")

    imgs = {
        'ROC Curve': f"{CFG.OUT_DIR}/oof_roc.png",
        'PR Curve': f"{CFG.OUT_DIR}/oof_pr.png",
        'Reliability': f"{CFG.OUT_DIR}/oof_reliability.png",
        'DET Curve': f"{CFG.OUT_DIR}/oof_det.png",
        'Score Histogram': f"{CFG.OUT_DIR}/oof_score_hist.png",
        'Class Balance': f"{CFG.OUT_DIR}/class_balance.png",
        'Augmentations': f"{CFG.OUT_DIR}/aug_preview_grid.png",
        'CM @ J*': f"{CFG.OUT_DIR}/cm_J_star.png",
        'CM @ Spec80': f"{CFG.OUT_DIR}/cm_spec80.png",
        'Robustness Δ': f"{CFG.OUT_DIR}/robustness_deltas.png",
        'Decision Curve': f"{CFG.OUT_DIR}/decision_curve.png",
    }
    gradcams = sorted(glob.glob(f"{CFG.OUT_DIR}/gradcam/*.png"))[:24]

    styles = "<style>body{font-family:system-ui,sans-serif;margin:2rem}h1,h2{margin-bottom:0.5rem}.grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:1rem}.card{border:1px solid #ddd;border-radius:8px;padding:1rem;box-shadow:0 2px 4px #0001}table{border-collapse:collapse;width:100%;margin-top:1rem}th,td{border:1px solid #ddd;padding:8px;text-align:left}th{background:#f7f7f7}</style>"
    kpis = []
    for k in ["thr_j","thr_s80","thr_85","global_t_temp"]:
        if k in thr: kpis.append(f"<div class='card'><b>{k}</b><br>{thr[k]:.4f}</div>")
    kpi_html = "".join(kpis)

    lat_html = ""
    if rep:
        lat_html = f"<table><tr><th>Mode</th><th>Mean (ms/img)</th><th>Throughput (img/s)</th></tr>" \
                   f"<tr><td>No-TTA</td><td>{rep['per_img_ms_no_tta']['mean']:.2f}</td><td>{rep['throughput_imgs_per_sec_no_tta']:.1f}</td></tr>" \
                   f"<tr><td>TTA</td><td>{rep['per_img_ms_tta']['mean']:.2f}</td><td>{rep['throughput_imgs_per_sec_tta']:.1f}</td></tr></table>"

    plots_html = "".join([f"<div class='card'><h3>{k}</h3><img src='{_img_data_uri(v)}' style='width:100%'></div>"
                          for k,v in imgs.items() if os.path.exists(v)])
    gradcam_html = "".join([f"<div class='card'><b>{os.path.basename(p)}</b><img src='{_img_data_uri(p)}' style='width:100%'></div>"
                            for p in gradcams])

    ci_html = ""
    if metrics_ci:
        rows = "".join([f"<tr><td>{k}</td><td>{v['mean']:.3f}</td><td>[{v['ci95'][0]:.3f}, {v['ci95'][1]:.3f}]</td></tr>" for k,v in metrics_ci.items()])
        ci_html = f"<h2>OOF 95% CIs</h2><table><tr><th>Metric</th><th>Mean</th><th>95% CI</th></tr>{rows}</table>"

    html = f"<!DOCTYPE html><html><head>{styles}<title>PCOS Appendix</title></head><body>"
    html += f"<h1>PCOS Appendix</h1><p>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>"
    if kpi_html: html += f"<h2>Key thresholds</h2><div class='grid'>{kpi_html}</div>"
    if lat_html: html += f"<h2>Latency</h2>{lat_html}"
    if ci_html:  html += ci_html
    html += f"<h2>Plots</h2><div class='grid'>{plots_html}</div>"
    if gradcam_html: html += f"<h2>Grad-CAM Examples</h2><div class='grid'>{gradcam_html}</div>"
    html += "</body></html>"

    with open(out_html, 'w') as f: f.write(html)
    print("[ok] appendix ->", out_html)

build_html_appendix_fileonly()


[ok] appendix -> /kaggle/working/pcosgen_out/appendix.html


In [11]:
# === Image-model metrics extractor (fills your comparison table) ===
import os, json, glob

OUT = "/kaggle/working/pcosgen_out"  # change if needed
def _load(p):
    try:
        with open(p) as f: return json.load(f)
    except Exception: return None

# find latency file (fast preferred)
lat = _load(os.path.join(OUT, "latency_report_fast.json")) or _load(os.path.join(OUT, "latency_report.json"))
ci  = _load(os.path.join(OUT, "oof_metrics_ci.json"))
thr = _load(os.path.join(OUT, "thresholds.json"))
cb  = _load(os.path.join(OUT, "class_balance.json"))

# pull values (robust to missing)
def get_latency(lat, mode):
    if not lat: return None
    rows = [r for r in lat.get("results", []) if r.get("mode")==mode]
    return rows[0] if rows else None

r1  = get_latency(lat, "no_TTA_bs1")
r8  = get_latency(lat, "no_TTA_bs8")
r32 = get_latency(lat, "no_TTA_bs32")

auc = ci.get("AUC_ROC", {}).get("mean") if ci else None
ap  = ci.get("AP", {}).get("mean") if ci else None
brier = ci.get("Brier", {}).get("mean") if ci else None

thr_j   = thr.get("thr_j") if thr else None
thr_s80 = thr.get("thr_s80") if thr else None
thr_85  = thr.get("thr_85") if thr else None
m_85    = thr.get("m_85") if thr else None  # contains F1 / Sens / Spec at the 85/85 target (or closest)

def fmt(x, digs=4):
    return "—" if x is None else (f"{x:.{digs}f}" if isinstance(x,(int,float)) else str(x))

# Build a compact “Image model” row you can paste into your paper table
row = {
    "AUC-ROC (OOF)": fmt(auc),
    "AP (PR-AUC)": fmt(ap),
    "Brier score": fmt(brier),
    "F1 @ 85/85": fmt(m_85.get("F1") if m_85 else None, 4),
    "Sens @ 85/85": fmt(m_85.get("RECALL_healthy") if m_85 else None, 4),
    "Spec @ 85/85": fmt(m_85.get("SPEC_unhealthy") if m_85 else None, 4),
    "J* threshold": fmt(thr_j, 3),
    "Spec80 threshold": fmt(thr_s80, 3),
    "Latency ms/img (bs=1, no-TTA)": fmt(r1.get("per_image_ms") if r1 else None, 2),
    "Throughput (bs=8, no-TTA) img/s": fmt(r8.get("throughput_img_per_s") if r8 else None, 2),
    "Throughput (bs=32, no-TTA) img/s": fmt(r32.get("throughput_img_per_s") if r32 else None, 2),
}

print("\n=== Image model (paste into your table) ===")
for k,v in row.items(): print(f"{k}: {v}")

# Optional: show class balance
if cb:
    print("\nClass balance:", cb.get("counts", {}), cb.get("percent", {}))
else:
    print("\nClass balance: (file not found; optional)")



=== Image model (paste into your table) ===
AUC-ROC (OOF): 0.8141
AP (PR-AUC): 0.6469
Brier score: 0.1757
F1 @ 85/85: 0.5980
Sens @ 85/85: 0.7752
Spec @ 85/85: 0.6787
J* threshold: 0.463
Spec80 threshold: 0.483
Latency ms/img (bs=1, no-TTA): 31.96
Throughput (bs=8, no-TTA) img/s: 121.43
Throughput (bs=32, no-TTA) img/s: 167.42

Class balance: {'0_unhealthy': 2297, '1_healthy': 903} {'0_unhealthy': 71.78125, '1_healthy': 28.218749999999996}


In [9]:
%%bash
if [ -d /kaggle/working/pcosgen_out ]; then
  rm -f /kaggle/working/pcosgen_artifacts.zip
  (cd /kaggle/working && zip -qr pcosgen_artifacts.zip pcosgen_out)
  ls -lh /kaggle/working/pcosgen_artifacts.zip
else
  echo "Missing /kaggle/working/pcosgen_out"
fi


-rw-r--r-- 1 root root 893M Aug 24 09:48 /kaggle/working/pcosgen_artifacts.zip
